In [ ]:
import sys
import os
import builtins

code_dir_root = "/home/jovyan/3MDBench/ds_processors/"
sys.path.insert(0, code_dir_root)
builtins.CODE_DIR_ROOT_ = code_dir_root
img_path_root = "/home/jovyan/3MDBench/data/IMAGEs/"
ds_path = os.path.join(img_path_root, "MSCOCO")
output_path_root = os.path.join(img_path_root, "generated")
import generation_script as gs

In [2]:
import json

sel_cap_file_path = os.path.join(ds_path, "sel_captions_5000.txt")
img_cap_file_path = os.path.join(ds_path, "image_caption_dict.txt")
captions_list_str = ""
with open(sel_cap_file_path, "r") as sel_cap_file, open(img_cap_file_path, "r") as img_cap_file:
    sel_cap_dict = json.load(sel_cap_file)
    img_cap_dict = json.load(img_cap_file)

In [3]:
### load raw_captions  import ast

prompt_path = os.path.join(ds_path, "generated")
raw_img_cap_file_path = os.path.join(prompt_path, "raw_img_caption_5000.txt")
# json_string = ""
# raw_img_cap_dict = dict()

# with open(raw_img_cap_file_path, "r") as raw_img_cap_file:
#     for line in raw_img_cap_file:
#         if not line.startswith(" counter:"):
#             json_string += line
#         else:
#             if len(raw_img_cap_dict) == 0:
#                 raw_img_cap_dict = json.loads(json_string)
#             else:
#                 raw_img_cap_dict |= json.loads(json_string)
#             json_string = ""
raw_img_cap_dict = gs.load_img_captions_dict(raw_img_cap_file_path)

In [4]:
### load para_captions 

import json
prompt_path = os.path.join(ds_path, "generated")
para_img_cap_file_path = os.path.join(prompt_path, "para_img_caption_5000.txt")
para_img_cap_dict = gs.load_img_captions_dict(para_img_cap_file_path)

In [32]:
### convert captioner_captions

import json
import numpy as np

def get_img_cap_list(captioner_img_cap_file_path):
    json_string = ""
    img_cap_list = []
    with open(captioner_img_cap_file_path, "r") as captioner_img_cap_file:
        for line in captioner_img_cap_file:
            if not line.startswith(" counter:"):
                json_string += line
            else:
                if len(img_cap_list) == 0:
                    img_cap_list = json.loads(json_string)
                else:
                    img_cap_list = [*img_cap_list, *(json.loads(json_string))]
                json_string = ""
    img_cap_arr = np.array(img_cap_list)
    img_cap_list = img_cap_arr.squeeze()
    
    return img_cap_list

prompt_path = os.path.join(ds_path, "generated")
img_cap_lists = np.array([])
for idx in range(1, 11, 1):
    captioner_img_cap_file_path = os.path.join(prompt_path, f"captioner_img_caption_5000_{idx}.txt")
    if img_cap_lists.shape[0] == 0:
        img_cap_lists = get_img_cap_list(captioner_img_cap_file_path)
    else:
        img_cap_list = get_img_cap_list(captioner_img_cap_file_path)
        img_cap_lists = np.vstack((img_cap_lists, img_cap_list))
captioner_img_caption_dict = dict()
img_ids = list(sel_cap_dict.keys())
for img_index in range(img_cap_lists.shape[1]):
    img_cap_arr = img_cap_lists[:, img_index]
    img_cap_arr = np.array([img_cap[1:] for img_cap in img_cap_arr], dtype=img_cap_arr.dtype)
    captioner_img_caption_dict[img_ids[img_index]] = img_cap_arr.tolist()
with open(os.path.join(prompt_path, "captioner_img_caption_5000.txt"), "w") as captioner_img_caption_file:
    json.dump(captioner_img_caption_dict, captioner_img_caption_file, indent=4)

In [3]:
### load captioner_captions 

import json
prompt_path = os.path.join(ds_path, "generated")
captioner_img_cap_file_path = os.path.join(prompt_path, "captioner_img_caption_5000.txt")
captioner_img_cap_dict = gs.load_cap_img_captions_dict(captioner_img_cap_file_path)

In [ ]:
### load kandinsky 3 pipeline

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
repo_id = "kandinsky-community/kandinsky-3"
variant= "fp16"
cache_dir = os.path.join(code_dir_root, "image_generators", "Kandinsky3", "cache")
t2i_or_i2i = False
pipe = gs.load_img_generator_pipeline(repo_id, variant, cache_dir, t2i_or_i2i, device)


In [ ]:
### load PixArt_Σ pipeline

import torch

repo_id = "PixArt-alpha/PixArt-Sigma-XL-2-1024-MS"
variant = ""
cache_dir = os.path.join(code_dir_root, "image_generators", "PixArt_Σ", "cache")
t2i_or_i2i = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch_compile = False
pipe = gs.load_img_generator_pipeline(repo_id, variant, cache_dir, t2i_or_i2i, device, torch_compile)

In [ ]:
### load StableDiffusion3 pipeline

import torch

repo_id = "stabilityai/stable-diffusion-3-medium-diffusers"
variant = ""
cache_dir = os.path.join(code_dir_root, "image_generators", "StableDiffusion3", "cache")
t2i_or_i2i = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch_compile = False
pipe = gs.load_img_generator_pipeline(repo_id, variant, cache_dir, t2i_or_i2i, device, torch_compile)

In [ ]:
from prompt_processors import image_similarity_metrics as ism
from PIL import Image
from torchvision.transforms.functional import pil_to_tensor
from image_generators.Kandinsky3 import generator as gen
import torch

size = (512, 512)
num_inference_steps = 50
img_width = 512
img_height = 512
count = 1
images = None
img_id = 0
output_path = os.path.join(output_path_root, "Kandinsky3", "MSCOCO", "raw_prompt")
for raw_key, raw_caps in raw_img_cap_dict.items():
    img_id = raw_key
    prompts = [*raw_caps, *raw_caps]
    if count < 2:
        real_img_path = os.path.join(ds_path, "selected", img_cap_dict[img_id][0])
        # images = gen.run_kandinsky3_t2i(pipe, num_inference_steps, img_width, img_height, prompts, 
                                        # manual_seed=False, seed=None)
        images = gen.run_kandinsky3_i2i(pipe, num_inference_steps, img_width, img_height, prompts, 
                                real_img_path, manual_seed=False, seed=None)
        
        real_img_tensor = gs.convert_img_to_imgtensor(real_img_path, None, size)
        similarities = torch.tensor([], dtype=torch.float32, device=device)
        for fake_img in images:
            fake_img_tensor = gs.convert_img_to_imgtensor(None, fake_img, size)
            similarity_metrics = torch.tensor([[
                                  ism.compute_LPIPS(real_img_tensor, fake_img_tensor, 'squeeze', 
                                                    device), 
                                  ism.compute_PSNR(real_img_tensor, fake_img_tensor, device),
                                  ism.compute_SSIM(real_img_tensor, fake_img_tensor, device)]],
                      dtype=torch.float32, device=device)
            similarities = torch.cat((similarities, similarity_metrics), 0)
        print(f"similarities: {similarities}")
        best_indices = torch.tensor([], dtype=torch.int32, device=device)
        for col_idx in range(similarities.size(dim=1)):
            if col_idx == 0:
                best_indices = torch.cat((best_indices, similarities[:,col_idx].min(dim=0)[1].unsqueeze(dim=0)), 0)
            else:
                best_indices = torch.cat((best_indices, similarities[:,col_idx].max(dim=0)[1].unsqueeze(dim=0)), 0)
        print(f"best_indices: {best_indices}")
        unique_counts = best_indices.unique(return_counts=True)[1]
        if (unique_counts == 1).all():
            unique_idx = torch.randint(0, unique_counts.size(dim=0), (1,))
            print(f"all 1s, unique_idx: {unique_idx}")
        else:
            unique_idx = unique_counts.max(dim=0)[1]
            print(f"unique max, unique_idx: {unique_idx}")
        print(f"unique_counts: {unique_counts}")
        result_img = images[best_indices[unique_idx]]
    else:
        break
    count += 1


In [39]:
output_path = os.path.join(output_path_root, "Kandinsky3", "MSCOCO", "raw_prompt")
output_file_path = os.path.join(output_path, "SSIM", img_cap_dict[img_id][0])
images[6].save(output_file_path)

In [ ]:
import os
import shutil

ds_path_root = "/home/jovyan/3MDBench/data/IMAGEs/"
real_path = f"{ds_path_root}MSCOCO/selected/"
fake_path = f"{ds_path_root}generated/Kandinsky3/MSCOCO/raw_prompt/"
out_path = os.path.join(fake_path, "REAL")
for _, _, img_files in os.walk(os.path.join(fake_path, "LPIPS")):
    for img_file in img_files:
        print(f"img_file: {img_file}")
        src_file_path = os.path.join(real_path, img_file)
        des_file_path = os.path.join(out_path, img_file)
        shutil.copy(src_file_path, des_file_path)


In [ ]:
for best_idx in range(best_indices.size(dim=0)):
            img_idx = best_indices[best_idx].item()
            if best_idx == 0:
                output_file_path_1 = os.path.join(output_path, "LPIPS", img_cap_dict[img_id][0])
                output_file_path_2 = os.path.join(output_path, f"LPIPS{count}", img_cap_dict[img_id][0])
            elif best_idx == 1:
                output_file_path_1 = os.path.join(output_path, "PSNR", img_cap_dict[img_id][0])
                output_file_path_2 = os.path.join(output_path, f"PSNR{count}", img_cap_dict[img_id][0])
            elif best_idx == 2:
                output_file_path_1 = os.path.join(output_path, "SSIM", img_cap_dict[img_id][0])
                output_file_path_2 = os.path.join(output_path, f"SSIM{count}", img_cap_dict[img_id][0])
            images[img_idx].save(output_file_path_1)
            images[img_idx].save(output_file_path_2)

In [3]:
import torch

a = torch.tensor([[[2,5,5], [3,6,9], [8,8,6]], [[2,3,5], [1,6,9], [2,1,1]]])
a.size()

torch.Size([2, 3, 3])

In [7]:
torch.repeat_interleave(a, 3, dim=2).size()

torch.Size([2, 3, 9])